In [ ]:
from roboflow import Roboflow
from PIL import Image
import os 
import json 
import numpy as np
import pandas as pd

In [ ]:
# object detection api documentation: https://docs.roboflow.com/deploy/hosted-api/custom-models/object-detection

api_key = "132cxQxyrOVmPD63wJrV" # api keys are individual, change to your own
model_endpoint = "" # model endpoint (after deployment)
version = 0 # version number

rf = Roboflow(api_key=api_key)
project = rf.workspace().project("MODEL_ENDPOINT")
model = project.version(version).model

In [ ]:
image_folder_path = "" # Your Image Folder Path Here
seal_images = [os.path.join(image_folder_path, file) for file in os.listdir(image_folder_path)]

Image.open(seal_images[0]) # for verification

In [ ]:
# adjust for model tendencies 
seal_conf = 20
seal_overlap = 30
clump_conf = 20
clump_overlap = 30

# extracting clumps 
clump_imgs = []
num_seals = [] # number of individual seals 

for img in seal_images:
    image = Image.open(img)

    seals = json.load(model.predict(img, confidence=seal_conf, overlap=seal_overlap, classes='seals').json())['predictions']
    num_seals.append(len(seals))

    clumps = json.loads(model.predict(img, confidence=clump_conf, overlap=clump_overlap, classes='clump').json())['predictions']

    # getting individual seals
    seal_pos = [] 
    for seal in seals:
        x1_seal = seal['x'] - seal['width'] / 2
        x2_seal = seal['x'] + seal['width'] / 2
        y1_seal = seal['y'] - seal['height'] / 2
        y2_seal = seal['y'] + seal['height'] / 2

        seal_pos.append((x1_seal, y1_seal, x2_seal, y2_seal))
    
    # getting clumps 
    for clump in clumps:
        x1_clump = clump['x'] - clump['width'] / 2
        x2_clump = clump['x'] + clump['width'] / 2
        y1_clump = clump['y'] - clump['height'] / 2
        y2_clump = clump['y'] + clump['height'] / 2

        top_left_clump = (x1_clump, y1_clump)
        bottom_right_clump = (x2_clump, y2_clump)

        for pos in seal_pos:
            if (max(x1_clump, pos[0]) <= min(x2_clump, pos[2])) & (max(y1_clump, pos[1]) <= min(y2_clump, pos[3])):
                seal_pos.remove(pos) 
        num_seals.append(len(seal_pos))

        subimage = image.crop((*top_left_clump, *bottom_right_clump))

        clump_imgs.append(subimage)

In [ ]:
# Extracting Length, Width and RGB metrics 

widths = []
heights = []
avg_r = []
sd_r = []
avg_g = []
sd_g = []
avg_b = []
sd_b = [] 

for clump in clump_imgs:
    
    width, height = clump.size

    widths.append(width)
    heights.append(heights)

    img_array = np.array(clump)

    avg_r.append(np.mean(img_array[1, :, :]))
    sd_r.append(np.sd(img_array[1, :, :]))
    avg_g.append(np.mean(img_array[:, 1, :]))
    sd_g.append(np.sd(img_array[:, 1, :]))
    avg_b.append(np.mean(img_array[:, :, 1]))
    sd_b.append(np.sd(img_array[:, :, 1]))

holistics = pd.DataFrame({'widths': widths, 
                          'heights': heights,
                          'avg_r': avg_r, 
                          'sd_r': sd_r, 
                          'avg_g': avg_g,
                          'sd_g': sd_g,
                          'avg_b': avg_b,
                          'sd_b': sd_b
                          })

In [ ]:
# writing holistics 

holistics.to_csv('holistics.csv') # change to filepath 

In [ ]:
# saving clumps as images in a new folder  

output_directory = "" # insert folder to save clumps 

num_imgs = len(clumps) # This could be a lot of images, so alter this object to save only a couple at a time if needed. 

for idx, img in enumerate(clumps[:num_imgs]):
    img_path = os.path.join(output_directory, f"clump_{idx+1}.png")
    img.save(img_path)